In [ ]:
import pandas as pd
import numpy as np
import math
import calendar
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.vector_ar.var_model import VAR

In [ ]:
store_calendar = pd.read_csv("Calendar_with_cycled_days.csv", index_col = 0)
date_converter = dict(zip(store_calendar['d'], store_calendar.index))

sell_prices = pd.read_csv("sell_prices_afcs2021.csv", index_col=0)

sample_submission = pd.read_csv("sample_submission_afcs2021.csv", index_col=0)

train_data = pd.read_csv("sales_train_validation_afcs2021.csv", index_col=0)
test_data = pd.read_csv("sales_test_validation_afcs2021.csv", index_col=0)
train_data = train_data.rename(columns=date_converter)
test_data = test_data.rename(columns=date_converter)

total_sales = train_data.sum()


# Attempt at forecasting

In [ ]:
big_train = train_data.merge(test_data, left_index=True, right_index=True)
big_train

In [ ]:
# Preprocess the train data
ts_train_data = big_train.transpose()
ts_train_data.index = pd.to_datetime(ts_train_data.index)
products = list(ts_train_data.columns.values)
originals = list(ts_train_data.columns.values)

for i in products:
    p_name = "_".join(i.split("_")[:3])
    ts_train_data = ts_train_data.rename(columns = {i : p_name})

ts_train_data.index.name = 'date'
ts_train_data

In [ ]:
# Reset index for merging
store_calendar = store_calendar.reset_index()

In [ ]:
events = store_calendar['event_name_1']
events_tf = np.zeros(len(events))

for i in range(len(events)):
    if isinstance(events[i], str):
        events_tf[i] = 1

events_tf = [int(item) for item in events_tf]
        
store_calendar['is_event'] = events_tf
store_calendar

In [ ]:
events = store_calendar['event_name_1']
unique_events = events.unique()[1:]
unique_events

for i in unique_events:
    arr = np.zeros(len(events))
    for j in range(len(events)):
        if events[j] == i:
            arr[j] = 1
    store_calendar[i] = [int(item) for item in arr]

store_calendar.columns.values[18:]
# ts_train_data.columns

In [ ]:
# Merge the calendar and sell_price dataframes
new = pd.merge(sell_prices, store_calendar, on='wm_yr_wk')
new = new.set_index('date')
new.index = pd.to_datetime(new.index)
new

In [ ]:
# Add noise to the sell price so it can be added to the VAR model (variables can't be constant)
original = new['sell_price']
noise = np.random.normal(.01, .001, len(new))
new['sell_price'] = new['sell_price'] + noise
new

In [ ]:
var_list = [['sin_wday', 'cos_wday', 'sell_price', 'is_event']]
events = store_calendar.columns.values[18:]
var_list.append(list(events))
var_list = [item for sublist in var_list for item in sublist]
var_list

In [ ]:
# Code in this cell from: https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/

from statsmodels.tsa.stattools import grangercausalitytests
maxlag=15
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df


df_train = ts_train_data[products[0]]
df_sales = new[new['item_id'] == products[0]]
dftest = pd.merge(df_sales, df_train, on="date")[[products[0], 'is_event', 'sin_month', 'cos_month', 'sell_price']]
print(dftest)

grangers_causation_matrix(dftest, variables = dftest.columns)

In [ ]:
# Code in this cell from: https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/

from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)


def adjust(val, length= 6): 
    return str(val).ljust(length)

cointegration_test(dftest)

In [ ]:
# Code in this cell from: https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/

from statsmodels.tsa.stattools import adfuller
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")
        
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

In [ ]:
# Create the forecasts
def make_forecast(train_data, price, product_names, original_product_names):
    submission = []
    
    for i in range(len(product_names)):
        df_train = train_data[product_names[i]]
        df_sales = price[price['item_id'] == product_names[i]]
        df = pd.merge(df_sales, df_train, on="date")[[product_names[i], 'sin_month', 'cos_month', 'sin_wday', 'cos_wday', 'is_event', 'sell_price']] #, 'SuperBowl', 
#                                                       'ValentinesDay', 'PresidentsDay', 'LentStart', 'LentWeek2', 
#                                                       'StPatricksDay', 'Purim End', 'OrthodoxEaster', 'Pesach End', 
#                                                       'Cinco De Mayo', "Mother's day", 'MemorialDay', 'NBAFinalsStart', 
#                                                       'NBAFinalsEnd', "Father's day", 'IndependenceDay', 'Ramadan starts', 
#                                                       'Eid al-Fitr', 'LaborDay', 'ColumbusDay', 'Halloween', 'EidAlAdha', 
#                                                       'VeteransDay', 'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear', 
#                                                       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter']]
        

#         Uncomment to make the data stationary
#         df = df.diff().dropna()
        
        model = VAR(df)
        model_fit = model.fit(maxlags=15, ic='bic')
        
#         Uncomment this to calculate the Durbin Watson statistic
#         out = durbin_watson(model_fit.resid)
#         for col, val in zip(df.columns, out):
#             print(adjust(col), ':', round(val, 2))
            
        prediction = model_fit.forecast(model_fit.y, steps=28)
        
#         Uncomment to de-difference the predictions
#         df_forecast = pd.DataFrame(prediction, columns=df.columns + '_2d')
#         df_fc = df_forecast.copy()
#         columns = df.columns
#         for col in columns: 
#             df_fc[str(col)+'_forecast'] = df[col].iloc[-1] + df_fc[str(col)+'_2d'].cumsum()

#         fc = df_fc[product_names[i]+'_forecast'].values.tolist()
        
        fc = prediction[:,0].tolist()
        fc.insert(0, originals[i])
        submission.append(fc)

    return submission
 
products = list(ts_train_data.columns.values)
df = make_forecast(ts_train_data, new, products, originals)

In [ ]:
# Convert forecasts to a dataframe
df = pd.DataFrame(df, columns=['id','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15','F16','F17','F18','F19','F20','F21','F22','F23','F24','F25','F26','F27','F28'])
df = df.set_index('id')
df

In [ ]:
# Calculate the RMSE
np.sqrt(mean_squared_error(df, test_data))

# Results

In [ ]:
# Save the best RMSE for comparison
best = df
np.sqrt(mean_squared_error(best, test_data))

In [ ]:
# Plot the actuals vs the forecasts of the first product in the dataset
plt.style.use('seaborn')
df1 = best.iloc[1]
df1.index = test_data.iloc[0].index
plt.plot(df1)
plt.plot(test_data.iloc[1])
plt.show()

In [ ]:
# Plot the actuals vs the forecasts of the average of all the products in the dataset
summed = df.sum(axis=0)/823
summed.index = test_data.iloc[0].index

plt.plot(summed)
plt.plot(test_data.sum(axis=0)/823)
plt.show()

# Make CSV file

In [ ]:
# Create the CSV file
sub = df.reset_index()
sub.to_csv("submission.csv", index=False)